In [17]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
#from _encode import _encode, _unique
from sklearn.preprocessing import OrdinalEncoder

In [18]:
prediction = 0
### Your Code Here ###
with open('ipl_csv2/all_matches.csv') as f:
    #ipl_data = pd.read_csv(f, low_memory=False)
    ipl_data = pd.read_csv(f, dtype={"match_id":int,"season":"string","start_date":"string","venue":"string",
                                     "innings":int,"ball":float,"batting_team":"string","bowling_team":"string",
                                     "striker":"string","non_striker":"string","bowler":"string","runs_off_bat":int,
                                     "extras":int,"wides":pd.Int64Dtype(),"noballs":pd.Int64Dtype(),
                                     "byes":pd.Int64Dtype(),"legbyes":pd.Int64Dtype(),"penalty":pd.Int64Dtype(),
                                     "wicket_type":"string","player_dismissed":"string",
                                     "other_wicket_type":"string","other_player_dismissed":"string"})

In [19]:
ipl_data

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
0,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,...,1,<NA>,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>
1,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.2,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.3,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,1,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.4,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.5,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,...,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196795,1254070,2021,2021-04-20,"MA Chidambaram Stadium, Chepauk, Chennai",2,18.6,Delhi Capitals,Mumbai Indians,Lalit Yadav,SO Hetmyer,...,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
196796,1254070,2021,2021-04-20,"MA Chidambaram Stadium, Chepauk, Chennai",2,18.7,Delhi Capitals,Mumbai Indians,SO Hetmyer,Lalit Yadav,...,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
196797,1254070,2021,2021-04-20,"MA Chidambaram Stadium, Chepauk, Chennai",2,18.8,Delhi Capitals,Mumbai Indians,SO Hetmyer,Lalit Yadav,...,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
196798,1254070,2021,2021-04-20,"MA Chidambaram Stadium, Chepauk, Chennai",2,19.1,Delhi Capitals,Mumbai Indians,SO Hetmyer,Lalit Yadav,...,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [20]:
relevantColumns = ['match_id', 'venue', 'innings', 'ball', 'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler','runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes','penalty']
ipl_data = ipl_data[relevantColumns]
ipl_data['total_runs'] = ipl_data['runs_off_bat'] + ipl_data['extras']
ipl_data = ipl_data.drop(columns=['runs_off_bat','extras'])
ipl_data = ipl_data[ipl_data['ball'] <= 5.6]
ipl_data = ipl_data[ipl_data['innings'] <= 2]
ipl_data = ipl_data.groupby(['match_id', 'venue', 'innings', 'batting_team', 'bowling_team', 'striker', 'bowler']).total_runs.mean()
ipl_data = ipl_data.reset_index()
ipl_data = ipl_data.drop(columns=['match_id'])
ipl_data['batting_team'] = ipl_data['batting_team'].replace(['Kings XI Punjab'],'Punjab Kings')
ipl_data['bowling_team'] = ipl_data['bowling_team'].replace(['Kings XI Punjab'],'Punjab Kings')
ipl_data['bowling_team'] = ipl_data['bowling_team'].replace(['Delhi Daredevils'],'Delhi Capitals')
ipl_data['batting_team'] = ipl_data['batting_team'].replace(['Delhi Daredevils'],'Delhi Capitals')
ipl_data['venue'] = ipl_data['venue'].replace(['M Chinnaswamy Stadium'],'M.Chinnaswamy Stadium')
ipl_data['venue'] = ipl_data['venue'].replace(['Rajiv Gandhi International Stadium, Uppal'],'Rajiv Gandhi International Stadium')
ipl_data['venue'] = ipl_data['venue'].replace(['Sardar Patel Stadium, Motera'],'Narendra Modi Stadium')
ipl_data['venue'] = ipl_data['venue'].replace(['Punjab Cricket Association IS Bindra Stadium, Mohali'],'Punjab Cricket Association IS Bindra Stadium')
ipl_data['venue'] = ipl_data['venue'].replace(['Punjab Cricket Association Stadium, Mohali'],'Punjab Cricket Association IS Bindra Stadium')
ipl_data['venue'] = ipl_data['venue'].replace(['Wankhede Stadium, Mumbai'],'Wankhede Stadium')
ipl_data['venue'] = ipl_data['venue'].replace(['Feroz Shah Kotla'],'Arun Jaitley Stadium')
ipl_data['venue'] = ipl_data['venue'].replace(['MA Chidambaram Stadium, Chepauk, Chennai'],'MA Chidambaram Stadium')
ipl_data['venue'] = ipl_data['venue'].replace(['MA Chidambaram Stadium, Chepauk'],'MA Chidambaram Stadium')

<ipython-input-20-dd4a5cd09c7c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ipl_data['total_runs'] = ipl_data['runs_off_bat'] + ipl_data['extras']


In [21]:
ipl_data

,venue,innings,batting_team,bowling_team,striker,bowler,total_runs
0,M.Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,AA Noffke,4.500000
1,M.Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,1.090909
2,M.Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,Z Khan,2.714286
3,M.Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,RT Ponting,Z Khan,0.000000
4,M.Chinnaswamy Stadium,1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,AA Noffke,1.666667
...,...,...,...,...,...,...,...
14138,MA Chidambaram Stadium,2,Delhi Capitals,Mumbai Indians,S Dhawan,TA Boult,1.000000
14139,MA Chidambaram Stadium,2,Delhi Capitals,Mumbai Indians,SPD Smith,J Yadav,0.857143
14140,MA Chidambaram Stadium,2,Delhi Capitals,Mumbai Indians,SPD Smith,JJ Bumrah,1.250000
14141,MA Chidambaram Stadium,2,Delhi Capitals,Mumbai Indians,SPD Smith,KH Pandya,1.000000


In [22]:
venue_encoder = LabelEncoder()
team_encoder = LabelEncoder()
player_encoder = OrdinalEncoder(handle_unknown='use_encoded_value',
                             unknown_value=600, dtype=int)
ipl_data['venue'] = venue_encoder.fit_transform(ipl_data['venue'])
ipl_data['batting_team'] = team_encoder.fit_transform(ipl_data['batting_team'])
ipl_data['bowling_team'] = team_encoder.fit_transform(ipl_data['bowling_team'])
ipl_data['striker'] = player_encoder.fit_transform(ipl_data[['striker']])
ipl_data['bowler'] = player_encoder.fit_transform(ipl_data[['bowler']])
ipl_data

,venue,innings,batting_team,bowling_team,striker,bowler,total_runs
0,15,1,5,12,37,17,4.500000
1,15,1,5,12,37,228,1.090909
2,15,1,5,12,37,351,2.714286
3,15,1,5,12,211,351,0.000000
4,15,1,5,12,227,17,1.666667
...,...,...,...,...,...,...,...
14138,16,2,2,6,217,323,1.000000
14139,16,2,2,6,241,124,0.857143
14140,16,2,2,6,241,135,1.250000
14141,16,2,2,6,241,158,1.000000


In [23]:
anArray = ipl_data.to_numpy()
anArray = anArray.astype(int)
anArray

array([[ 15,   1,   5, ...,  37,  17,   4],
       [ 15,   1,   5, ...,  37, 228,   1],
       [ 15,   1,   5, ...,  37, 351,   2],
       ...,
       [ 16,   2,   2, ..., 241, 135,   1],
       [ 16,   2,   2, ..., 241, 158,   1],
       [ 16,   2,   2, ..., 241, 323,   1]])

In [24]:
X,y = anArray[:,:6], anArray[:,6]
X

array([[ 15,   1,   5,  12,  37,  17],
       [ 15,   1,   5,  12,  37, 228],
       [ 15,   1,   5,  12,  37, 351],
       ...,
       [ 16,   2,   2,   6, 241, 135],
       [ 16,   2,   2,   6, 241, 158],
       [ 16,   2,   2,   6, 241, 323]])

In [25]:
y

array([4, 1, 2, ..., 1, 1, 1])

In [26]:
X = np.concatenate((np.eye(42)[anArray[:,0]], np.eye(2)[anArray[:,1]-1], np.eye(15)[anArray[:,2]],
                    np.eye(15)[anArray[:,3]], np.eye(601)[anArray[:,4]], np.eye(601)[anArray[:,5]],), axis = 1)
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [27]:

X_train , X_test, y_train, y_test = train_test_split(X,y,test_size = 0.25)
clf = RandomForestClassifier(n_estimators=100, random_state=None)
#clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)
#linearRegressor = LinearRegression()
#linearRegressor.fit(X_train, y_train)
test_case = pd.read_csv('inputFile.csv')
batsmen = test_case['batsmen'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('batsmen')
bowlers = test_case['bowlers'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('bowlers')
test_case = test_case.drop('batsmen', axis=1).join(batsmen)
test_case = test_case.drop('bowlers', axis=1).join(bowlers)
test_case['venue'] = venue_encoder.transform(test_case['venue'])
test_case['batting_team'] = team_encoder.transform(test_case['batting_team'])
test_case['bowling_team'] = team_encoder.transform(test_case['bowling_team'])
test_case['batsmen'] = player_encoder.transform(test_case[['batsmen']])
test_case['bowlers'] = player_encoder.transform(test_case[['bowlers']])
test_case

,venue,innings,batting_team,bowling_team,batsmen,bowlers
0,0,1,13,0,600,600
0,0,1,13,0,600,600
0,0,1,13,0,600,600
0,0,1,13,0,600,600
0,0,1,13,0,600,600
0,0,1,13,0,600,600
0,0,1,13,0,600,600
0,0,1,13,0,600,600
0,0,1,13,0,600,600
0,0,1,13,0,600,600


In [28]:

#test_case = test_case[['venue', 'innings', 'batting_team', 'bowling_team']]
testArray = test_case.to_numpy()
testArray


array([[  0,   1,  13,   0, 600, 600],
       [  0,   1,  13,   0, 600, 600],
       [  0,   1,  13,   0, 600, 600],
       [  0,   1,  13,   0, 600, 600],
       [  0,   1,  13,   0, 600, 600],
       [  0,   1,  13,   0, 600, 600],
       [  0,   1,  13,   0, 600, 600],
       [  0,   1,  13,   0, 600, 600],
       [  0,   1,  13,   0, 600, 600],
       [  0,   1,  13,   0, 600, 600],
       [  0,   1,  13,   0, 600, 600],
       [  0,   1,  13,   0, 600, 600]], dtype=int64)

In [29]:
test_case = np.concatenate((np.eye(42)[testArray[:,0]], np.eye(2)[testArray[:,1]-1], np.eye(15)[testArray[:,2]],
                    np.eye(15)[testArray[:,3]], np.eye(601)[testArray[:,4]], np.eye(601)[testArray[:,5]],), axis = 1)
test_case

array([[1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       ...,
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.]])

In [30]:
#prediction = linearRegressor.predict(test_case)
prediction = clf.predict(test_case)
print("Prediction Array: ", prediction)

Prediction Array:  [0 0 0 0 0 0 0 0 0 0 0 0]


In [32]:
total = np.sum(prediction)

In [33]:
total

0